![BroadWeir.bmp](BroadWeir.bmp)

In [1]:
%reset -f

from ipywidgets import *
import ipympl
import matplotlib.pyplot as plt

import sys
import proteus
## Required imports
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so, Comm, Context
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread

sys.path.append('/home/jovyan/jamInternship/air-water-vv/2d/hydraulicStructures/broad_crested_weir_GUI')


import broad_crested_weir as plant
reload(plant)
import broad_crested_weir_so as plant_so
reload(plant_so)
#


import numpy as np
from IPython.display import clear_output, display,FileLink,FileLinks

from tables import  openFile
from collections import  namedtuple



#import PostProcessing 
#reload(PostProcessing)
import helpers
reload(helpers)
import utilities
reload(utilities)
import PostProcessing
reload(PostProcessing)

from IPython.core.display import HTML
import webbrowser

global simulation_thread, ns

plt.close("all")

# Create Tank Dimension Widgets   

tankLength=FloatSlider(min=0.0, max=10.0, step=0.05, continuous_update=False,value=plant.tank_dim[0],
                  description='Tank Length:')
tankHeight=FloatSlider(min=0.1, max=3.0, step=0.05, continuous_update=False,value=plant.tank_dim[1],
                  description='Tank Height:')
obsWidth=widgets.FloatRangeSlider(
                                value=[plant.obstacle_x_start, plant.obstacle_x_end],
                                min=0.0, max=3.0, step=0.05,
                                description='Obs Range:',
                                continuous_update=False,
                                slider_color='white',
                                color='black'
                            )
obsHeight=FloatSlider(min=0.1, max=3.0, step=0.05, continuous_update=False,value=plant.obstacle_height,
                  description='Obs Height')
waterZ=FloatSlider(min=0.0, max=3.0, step=0.05, continuous_update=False,value=plant.opts.water_level,
                      description='Water Level:')
waterX=FloatSlider(min=0.1, max=10, step=0.05, continuous_update=False,value=plant.opts.water_width_over_obst,
                      description='Water width over obstacle:')     
outflow=FloatSlider(min=0.1, max=0.6, step=0.01, continuous_update=False,value=plant.outflow_level,
                      description='Outflow Level:')  


tankBox=VBox([tankLength,tankHeight,])
obsBox=VBox([obsWidth,obsHeight])
waterBox=VBox([waterZ,waterX,outflow])


f1 = plt.figure()
plt.plot([])
page1Controls=HBox([tankBox,obsBox,waterBox])
page1=VBox([page1Controls,f1.canvas])


def on_value_change(change):
    clear_output()
    Length=tankLength.value
    Height=tankHeight.value
    obsW=obsWidth.value
    obsH=obsHeight.value
    waterLine_z=waterZ.value
    waterLine_x=waterX.value
    outflow_level=outflow.value
    
    #Change Context
    plant.opts.water_level=waterLine_z
    plant.opts.water_width_over_obst=waterLine_x 
    plant.opts.outflow_level=outflow_level
    plant.opts.tank_dim=(Length,Height)
    plant.opts.obstacle_dim=(obsW[1]-obsW[0],obsH)
    plant.opts.obstacle_x_start=obsW[0]
    plant.Update_Model()
    Context.setFromModule(plant,mutable=True)
    ct = Context.get()
    plant_so.ct=ct
    #Plot new tank
    helpers.f=plt.figure(num=1)
    helpers.plot_domain()
    
   

tankLength.observe(on_value_change, names='value')
tankHeight.observe(on_value_change, names='value')
obsWidth.observe(on_value_change, names='value')
obsHeight.observe(on_value_change, names='value')
waterZ.observe(on_value_change, names='value')
waterX.observe(on_value_change, names='value')
outflow.observe(on_value_change, names='value')
 
on_value_change(True)


# Link widget values with the current tank dimension values
mylink1 = jslink((tankLength, 'value'), (obsWidth, 'max'))
mylink2 = jslink((tankHeight, 'value'), (obsHeight, 'max'))
mylink3 = jslink((tankLength, 'value'), (waterZ, 'max'))
mylink4 = jslink((tankHeight, 'value'), (waterX, 'max'))

#============================================================================================================================
#Physics and Numerics
#Load the modules the define the equations to be solved and the numerical methods to use.
#============================================================================================================================


#Prepare Widgets
txb_refine=IntText(value=plant.opts.refinement,step=1,
                        description='Refinement')
txb_he=FloatText(value=plant.opts.he, step=1e-15,
                    description='he', disabled=False)
showTriangles=widgets.Checkbox(value=True, description='Show Triangles', disabled=False)
txb_dtInit=FloatText(value=plant.opts.dt_init,step=.001,
                        description='dt initial',disabled=False)
txb_dtFixed=FloatText(value=plant.opts.dt_fixed,step=.001,
                       description='dt')
txb_tf=FloatText(value=plant.opts.T,step=.001,
                        description='Simulation Time')

btn_parameters = widgets.Button(description="Run Simulation")

#Button simulate code
def on_button_clicked(b):
    global simulation_thread, ns
    clear_output()
    print("Computing Parameters...")
    !rm mesh.*
    #plant.opts.refinement = txb_refine.value
    plant.opts.dt_init = txb_dtInit.value
    plant.opts.dt_fixed = txb_dtFixed.value
    plant.opts.T = txb_tf.value
    plant.opts.he = txb_he.value
    plant.Update_Model()
    plant.triangleOptions="VApq30Dena%8.8f" % ((txb_he.value**2)/2.0,)
    Context.setFromModule(plant,mutable=True)
    ct = Context.get()
    plant_so.ct=ct
    
    #attrs = vars(ct.domain.MeshOptions)
    #for item in attrs.items():
    #    print "attr: %s \n value:%s" % item 

    so = plant_so
    #so.tnList = [0.0,plant.dt_init]+[i*plant.dt_fixed for i in range(1,plant.nDTout+1)]
    so.tnList = [0.0]+[i*plant.dt_fixed for i in range(1,plant.nDTout+1)]
    info = open("TimeList.txt","w")
    for time in so.tnList:
        info.write(str(time)+"\n")
    info.close()
    plant_so.tnList=so.tnList
    pList=[]
    nList=[]
    so.sList=[]
    OptDB = PETSc.Options()
    for (p,n) in so.pnList:
        so.sList.append(default_s)
        pList.append(__import__(p))
        reload(pList[-1])
        nList.append(__import__(n))
        reload(nList[-1])
        pList[-1].name = p
        nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
        nList[-1].levelLinearSolver = default_n.KSP_petsc4py
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
    opts.save_dof = True
    opts.dataDir='.'
    opts.probDir='.'
    opts.logLevel=7
    opts.verbose=True
    opts.viewMesh=True
    
    #-----------------------------------------------------------------------
    ## Numerical Solution Object
    #    Now we create an the numerical wavetank object and set it up to run in a thread on each engine.
    #-----------------------------------------------------------------------
    !rm *.h5
    !rm *.xmf
    print("Creating Numerical Solution Object...")
    import time
    t1=time.time()
    ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)
    # Create a thread wrapper for the simulation.  The target must be an argument-less
    # function so we wrap the call to `calculateSolution` in a simple lambda:
    print("Creating Simulation Thread...")
    simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
    print("Starting Thread and Simulation Monitor")
    simulation_thread.start()
    f2.clear()
    plt.plot([])
    f2.canvas.draw()
    utilities.simulation_thread=simulation_thread
    utilities.ns=ns
    utilities.plt=plt
    utilities.f=plt.figure(num=2)
    utilities.plant=plant
    utilities.showTriangles=showTriangles.value
    utilities.monitor_simulation(refresh=5.0)
#
    print("Complete in:")
    t2=time.time()
    print(t2-t1)

    
f2=plt.figure()
f2.clear()
plt.plot([])
f2.canvas.draw()
    
btn_parameters.on_click(on_button_clicked)
Sim_Controls=VBox([txb_tf,
                   txb_dtFixed,
                   txb_dtInit,
                   txb_he,
                   #txb_refine,
                   showTriangles,
                   btn_parameters])
page3=HBox([Sim_Controls,f2.canvas])



#============================================================================================================================
# Create Post-Processing
#============================================================================================================================
f3=plt.figure()
f3.clear()
plt.plot([])
f3.canvas.draw()
 

btn_createImages=Button(description='Create Images')

def on_btn_createImages_clicked(b):
    clear_output()
    print("Creating Images ...")
    f3.clear()
    !rm phi*png
    PostProcessing.plant=plant
    PostProcessing.plt=plt
    PostProcessing.f=plt.figure(num=3)
    PostProcessing.plant_so=plant_so
    PostProcessing.Create_Images('broad_crested_weir')
    clear_output()
    f3.canvas.draw()
    print('Images Created')

btn_createImages.on_click(on_btn_createImages_clicked)
    
btn_createVideo=Button(description='Create Video',)
    
def on_btn_createVideo_clicked(b):
    clear_output()
    rate=int(round(1/txt_dtFixed))
    !rm -f plant.mp4; LD_LIBRARY_PATH='' avconv -i -r {rate} phi%4d.png -vcodec libx264 plant.mp4 -loglevel quiet
    PostProcessing.Create_Video()
    clear_output()
    print("Click link to see video:")
    display(FileLink('plant.mp4'))
    
btn_createVideo.on_click(on_btn_createVideo_clicked)

page4=HBox([VBox([btn_createImages,btn_createVideo]), f3.canvas])


#============================================================================================================================
# Widget Display
#============================================================================================================================

# Display Widgets in Tabs
lst=['Bathmetry and SDF','Simulation','Post Processing']
children = [page1,page3,page4]
tab = widgets.Tab(children=children)
[tab.set_title(num, name) for num, name in enumerate(lst)]
display(tab)



Computing Parameters...
File already exists, not writing polyfile: 'mesh'
Creating Numerical Solution Object...
Creating Simulation Thread...
Starting Thread and Simulation Monitor
u_max=0.139, v_max=0.000, Vmax=0.139, cfl=0.000, dt=1.0000000000
Monitored for: 0:00:05.701726. at t= 2.50000e-02
u_max=1.163, v_max=2.251, Vmax=2.952, cfl=3.785, dt=0.0250000000
Monitored for: 0:00:21.889378. at t= 2.98656e-02
u_max=1.023, v_max=1.980, Vmax=2.621, cfl=0.720, dt=0.0053499083
Monitored for: 0:00:38.899227. at t= 5.00000e-02
u_max=1.318, v_max=2.015, Vmax=2.489, cfl=0.736, dt=0.0059310567
Monitored for: 0:00:53.270958. at t= 6.81561e-02
u_max=1.550, v_max=2.045, Vmax=2.390, cfl=0.398, dt=0.0034219381
Monitored for: 0:01:14.925378. at t= 8.80682e-02
u_max=1.846, v_max=1.993, Vmax=2.277, cfl=0.736, dt=0.0067056119
Monitored for: 0:01:30.272531. at t= 1.06955e-01
u_max=2.058, v_max=1.845, Vmax=2.187, cfl=0.737, dt=0.0070453046
Monitored for: 0:01:44.756712. at t= 1.21174e-01
u_max=2.221, v_max=1.